# ファイル操作

In [1]:
const
fs = require("fs"), // 又は require("fs-extra")
os = require("os"),
process = require("process"),
child_process = require("child_process");

Node.js ではファイル操作をする関数が同期処理用と非同期処理用の2つが用意されている。
非同期処理用のものは,その関数を実行した直後の行から,関数内の処理内容と並列的に進行する。関数を実行し終えた後に実行したい内容は,関数にコールバック関数として渡す。
同期処理用のものは,その関数を実行した時点で,その直後の行以降の内容は一旦中断される。関数を実行し終えてから直後の行から実行を再開する。
```JavaScript
mkdir(path,callbackFn) // 非同期処理用関数
mkdirSync(path) // 同期処理用関数
```

`fs-extra` は `fs` の上位互換 (`fs` の機能は全て `fs-extra` に含まれている) であり,こちらを使うことで, `fs` にない機能が提供される。  
しかし,こちらは標準搭載されていないので, `npm install fs-extra` でインストールする必要がある。  
以後, `fs-extra` が必要な機能は `only in fs-extra` と表記する

* カレントディレクトリをホームに変更

In [2]:
process.chdir(os.homedir());

## ファイル/フォルダ/シンボリックリンクの作成

* Emptyという空フォルダを作成

In [3]:
fs.mkdirSync("Empty");

* 一時ディレクトリの場合
```JavaScript
let t=os.tmpdir();
path.join(t,fs.mkdtempSync(process.pid)) // ディレクトリのパス
```

* Blankという空ファイルを作成

In [4]:
// writeFileは既にファイルが存在していれば置換する。
fs.writeFileSync("Blank","");

* フォルダEmptyの中にファイルBlankのシンボリックリンクSymlinkを作成

In [5]:
fs.symlinkSync("Blank","Empty/Symlink");

- ハードリンクの場合
```JavaScript
fs.linkSync("Blank","Empty/Symlink");
```

- `fs.symlinkSync(target,dst,type)`  
	`type` はWindowsのみ影響のある引数 (他のOSでは無視される)
	`"file"`,`"dir"`,`"junction"` のうちから選択する
	`"file"`,`"dir"` の場合は引数がなくても `target` 引数の指し示すものから自動的に判別する

## 書込み
* Untitled.mdというMarkdownファイルを作成して書込み

In [6]:
fs.writeFileSync("Untitled.md","# Header 1");

## 移動/名称変更

* フォルダEmptyをPackageに名称変更

In [7]:
fs.renameSync("Empty","Package");

* Packageフォルダ内のSymlinkファイルをAliasに名称変更

In [8]:
fs.renameSync("Package/Symlink","Package/Alias");

* Untitled.mdを移動して,名称変更

In [9]:
fs.renameSync("Untitled.md","Package/Headers.md");

## 追記
* Markdownファイルに追記

In [10]:
fs.appendFileSync(
	"Package/Headers.md",
	os.EOL+"## Header 2"+os.EOL+"### Header 3"
);

## 読込み
* Markdownファイルを読込み

In [11]:
fs.readFileSync("Package/Headers.md","utf-8");

'# Header 1\n## Header 2\n### Header 3'

- オプションで `"utf-8"` を指定しないとバッファデータが返される

## 再帰的にフォルダ作成
* フォルダを一気に作成

In [12]:
fs.mkdirSync("Package/Module/Submodule/Item",{recursive:true});
// fs.mkdirsSync("Package/Module/Submodule/Item"); // only in fs-extra

'Package/Module'

- `mkdirSync` のオプションで `recursive:true` とすることで,作成するフォルダItemの上位フォルダModule,Submoduleが存在していなくても,同時に生成される

## ディレクトリの内容を表示

In [13]:
fs.readdirSync("Package");

[ 'Alias', 'Headers.md', 'Module' ]

- ワイルドカードを使って,条件を満たすファイルを見つけ出す場合は

```JavaScript
const glob=require("glob");
glob.sync("*.md");
```
* 非同期の場合は `glob("*.md",callbackFn)` とする
* `glob` は標準ではないので, `npm install glob` でインストールする

## ファイル/フォルダの複製
* BlankとModuleを複製

In [14]:
fs.copyFileSync("Blank","Package/Blank");
// fs.copySync("Package/Module","Package/Module Copy"); // only in fs-extra

- `fs.copySync` を用いることで,フォルダ内の内容もまとめて複製できる

## ファイル/フォルダの削除

* ファイルBlankを削除

In [15]:
fs.unlinkSync("Blank");

* シンボリックリンクAliasを削除

In [16]:
fs.unlinkSync("Package/Alias");

* フォルダModuleを削除

In [17]:
// fs.removeSync("Package/Module"); // only in fs-extra

- 空のフォルダは `fs.rmdirSync` で削除できる。空でない場合は削除できない。  
	`FileUtils.rm_rf` を使えば,空でなくても削除できる
	`fs.rmdirSync` をファイルやシンボリックリンクに, `fs.unlinkSync` をフォルダに使うことはできない

## 権限を確認/変更

* 状態を確認する関数

In [18]:
let check=pt=>{

	console.log(fs.existsSync(pt) ? "存在しています" : "存在していません");

	if (fs.existsSync(pt)) {
		let stat=fs.statSync(pt);
		if (stat.isSymbolicLink()) {
			console.log("シンボリックリンクです");
			console.log(`リンク先: ${fs.readlinkSync(pt)}`);
			if (stat.isFile()) console.log("リンク先はファイルです");
			if (stat.isDirectory()) console.log("リンク先はフォルダです");
		}
		else {
			if (stat.isFile()) console.log("ファイルです");
			if (stat.isDirectory()) console.log("フォルダです");
		}
		if (stat.mode & fs.constants.S_IRUSR) console.log("読込可能です");
		if (stat.mode & fs.constants.S_IWUSR) console.log("書込可能です");
		if (stat.mode & fs.constants.S_IXUSR) console.log("実行可能です");
	}

};

* Markdownファイルの現在の状態を確認

In [19]:
check("Package/Headers.md");

存在しています
ファイルです
読込可能です
書込可能です


* 読込/書込権限を剥奪

In [20]:
fs.chmodSync("Package/Headers.md",0o044);

* 状態を確認

In [21]:
check("Package/Headers.md");

存在しています
ファイルです


* 読込/実行権限を付加

In [22]:
fs.chmodSync("Package/Headers.md",0o544);

* 状態を確認

In [23]:
check("Package/Headers.md");

存在しています
ファイルです
読込可能です
実行可能です


- `fs.chmodSync` は数値により権限を指定する
	* 4: 読込可能 (Readable)
	* 2: 書込可能 (Writable)
	* 1: 実行可能 (eXecutable)

- これらの和を3つ並べて指定する。  
    1つ目はユーザの権限,2つ目はゲストの権限,3つ目はその他の者の権限である。  
	JavaScriptでは8進数で表記するために,前に `0o` を付加する

	e.g. `fs.chmodSync(somefile,0o754)`
	* ユーザは 7 = 4+2+1 だから,読込可能,書込可能,実行可能
	* ゲストは 5 = 4 + 1 だから,読込可能,実行可能
	* その他は 4 だから,読込のみ可能